<center>
<br />
<h1>Code optimization, Numpy, cython</h1>
<h3>Python</h3>
<br />
<h4>2020</h4> </center>

# Matrix

Create Matrix class:

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

The simplest implementation of matrix multiplication:

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

Let's check its correctness:

In [ ]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


How much time does it take?

In [ ]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [ ]:
%%timeit
matrix_product(*setup())

Woah! Almost an entire second! That's too long. <br>
Let's find out where we waste time, for that we can use line_profiler:

In [ ]:
# !pip3 install line_profiler

# If failed:
# git clone https://github.com/rkern/line_profiler.git
# find line_profiler -name '*.pyx' -exec cython {} \;
# cd line_profiler && pip3 install . --user 

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f matrix_product matrix_product(*setup())

```
Timer unit: 1e-06 s

Total time: 2.33726 s
File: <ipython-input-4-b9cb39bb9b1b>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2         1         13.0     13.0      0.0      n_xrows, n_xcols = X.shape
     3         1          4.0      4.0      0.0      n_yrows, n_ycols = Y.shape
     4                                               
     5         1          2.0      2.0      0.0      assert n_xcols == n_yrows, "Incompatible matrix dimensions"
     6                                               
     7         1        218.0    218.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     8                                               
     9       101         76.0      0.8      0.0      for i in range(n_xrows):
    10     10100       7109.0      0.7      0.3          for j in range(n_xcols):
    11   1010000     740164.0      0.7     31.7              for k in range(n_ycols):
    12   1000000    1589675.0      1.6     68.0                  Z[i][k] += X[i][j] * Y[j][k]
    13         1          1.0      1.0      0.0      return Z
```

We see that a lot of time is spent on for-loop and getting indexes. What do you think can be improved?

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [ ]:
%%timeit
matrix_product(*setup())

Let's try to remove some element retrieval by index:

In [ ]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [ ]:
%%timeit
matrix_product2(*setup())

2 times faster! Since it became about 2 times less operations of retrieval by index.

In [ ]:
%lprun -f matrix_product2 matrix_product2(*setup())

```
Timer unit: 1e-06 s

Total time: 2.01848 s
File: <ipython-input-12-e24c1e2d67bf>
Function: matrix_product2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product2(X, Y):
     2         1         11.0     11.0      0.0      n_xrows, n_xcols = X.shape
     3         1          3.0      3.0      0.0      n_yrows, n_ycols = Y.shape
     4         1        109.0    109.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5       101         73.0      0.7      0.0      for i in range(n_xrows):
     6       100        102.0      1.0      0.0          Xi = X[i]
     7     10100       7580.0      0.8      0.4          for k in range(n_ycols):
     8     10000       7562.0      0.8      0.4              acc = 0
     9   1010000     738306.0      0.7     36.6              for j in range(n_xcols):
    10   1000000    1254721.0      1.3     62.2                  acc += Xi[j] * Y[j][k]
    11     10000      10016.0      1.0      0.5              Z[i][k] = acc
    12         1          1.0      1.0      0.0      return Z
```

Let's go further and try to be even faster!

In [ ]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [ ]:
X = [[1],[2],[3]]
Y = [[4,5,6]]
Z = [[0,0,0],[0,0,0],[0,0,0]]
Yt = [[4],[5],[6]]

for i, (Xi, Zi) in enumerate(zip(X, Z)):
        print((i,Xi,Zi))
        for k, Ytk in enumerate(Yt):
            print((k,Ytk))

In [ ]:
%%timeit
matrix_product3(*setup())

Again almost 2 times faster!

Conclusion: if your code is slow, do not rush to use other frameworks. Often you can speed up your code at times with little manipulation.

# Numpy

Let's try to take our first implementation, but use an array from numpy instead (we'll see how it works later)

In [ ]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [ ]:
%timeit np_matrix_product(*setup_np())

No profit so far.

In [ ]:
X, Y = setup_np()

In [ ]:
%timeit np.dot(X, Y)

It's getting interesting. Good motivation to learn the numpy module.

## Array

In [ ]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)

You can see that the array is effectively stored in memory:

In [ ]:
print(a.nbytes)
a = np.array([4,1.1,2])
print(a.nbytes)

Shape can be changed:

In [ ]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

a.shape = (2, 2)
a

An array is of one type. If you create differently typed elements, they all will be cast to the same type.

In [ ]:
np.array([4, 1.1, 2, 3, "string"])

## UFuncs

UFuncs (universal functions) are functions that work element by element.

In [ ]:
a = range(4)
b = [value + 1 for value in a]
print(b)

In numpy:

In [ ]:
a = np.arange(4)
b = a + 1
print(b)

The addition of two arrays.

In [ ]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for (i, j) in zip(a, b)]

In [ ]:
a, b = np.array(a), np.array(b)
%timeit a + b

It's because numpy is written in c.

Other UFuncs

In [ ]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

In [ ]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

There are many other UFuncs: arithmetic operations, bitwise, comparisons, trigonometric, etc.

## Aggregation

Numpy has many built-in aggregation functions

<img src="aggregations.png" width=600px>

In [ ]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

In [ ]:
np_data = np.array(data)
%timeit np.min(np_data)

## Indexing

There are all basic slices and indexes. But there are many additional tasty features:

In [ ]:
X = np.arange(6).reshape((2, 3))
X

In [ ]:
X[1]

In [ ]:
X[0, 1]

In [ ]:
z = X[:1, 1:]
z

Unlike with standard lists, numpy slices are not copies and you can change the initial list.

In [ ]:
z[:1, :1] = 100

In [ ]:
X

### Masking

In [ ]:
a = np.arange(8) ** 2
a[a > 8]

This works because numpy.array supports boolean mask indexing:

In [ ]:
a > 8

You can also use an array of indices:

In [ ]:
a = np.arange(4) * 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

## Broadcasting
Or automatic casting of dimensions.

You can stack arrays of different sizes where it makes sense:

In [ ]:
a = np.arange(3)

In [ ]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

In [ ]:
b + a

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

But it does not always work as we would like:

In [ ]:
X = np.arange(6).reshape((3, 2))
X

In [ ]:
X + np.array([1, 2, 3])

Of course, we understand what was meant, but numpy did not understand. We will tell him:

In [ ]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # reorient array
print(Y.shape)
print(Y)

In [ ]:
X + Y

Conclusion: Loops in python are slow. It is worth using the built-in numpy functions soo the code is more understandable and significant acceleration is gained.

# Cython 
Yet another way to optimize your code

The Cython compiler translates a program written in Python or Cython into more efficient C code

Cython is a Python add-on, supporting C function calls and assignment C language data type variables

Let's see how it works. Let's create a couple of functions in pure python:

In [ ]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f(0, 10, 1000000)

Enabling Cython magic:

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a 
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython is trying to determine variable types and due to this speed up. But those lines of code where he could not establish the type are highlighted in yellow. Almost everything is yellow.

In [ ]:
%%timeit
integrate_f2(0, 10, 1000000)

It's a bit faster, but we can do better.

Let's apply static typing:

In [ ]:
%%cython -a 
def f(double x):
    return x**2

def integrate_f3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f3(0, 10, 1000000)

Function calls in python can be expensive. For example, in the last example, there was a recurrent conversion from C double to Python float. 

Let's change the prototype of the function f:

In [ ]:
%%cython -a 
cdef double f2(double x):
    return x**2

cdef double integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f2(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f4(0, 10, 1000000)

Let's compare with numpy

In [ ]:
%%timeit
x = np.linspace(0, 10, 1000000)
(x**2).sum() / 100000

With Cython you can achieve more than with numpy!

## Matrix
Back to our Matrix example

In [ ]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


The whole thing is yellow!

In [ ]:
shape = (100, 100)

In [ ]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [ ]:
%timeit cy_matrix_product(X, Y)

It is faster than our Python implementation, but so far it does not reach the numpy version. <br>
Let's try to tell cython what types we use, maybe it will be faster.

In [ ]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [ ]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product(X, Y)

Now it's worse!

Cython can't work with Python imports and now it's required to ask python for every little snippet of code.

Let's explain data types for Cython explicitly.

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product2(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit  cy_matrix_product2(X, Y)

It seems that cython now understands more, but it's still slow, since the main parts are still not clear.

Let's add static typing for input values - now our function takes only values of a specified type.

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np  # Not every module may be imported like this


def cy_matrix_product3(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product3(X, Y)

Well, now we’ll add quite a bit of black magic. <br>
Add decorators that check the boundaries of arrays and type overflows.

In [ ]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product4(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product4(X, Y)

Profit.

# Memory Usage (just in case)

In [ ]:
#!pip install memory_profiler

In [ ]:
%load_ext memory_profiler

## 1) memit

In [ ]:
%%memit
a = [0] * 2*10**8
del a

## 2.1) jupyter mprun

In [ ]:
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

%mprun -T mprof0 -f my_func my_func()

In [ ]:
%%writefile memscript.py
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

In [ ]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()

In [ ]:
!cat mprof0

## 2.2) Terminal mprun

In [ ]:
%%writefile memscript2.py
@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()

In [ ]:
!cat memscript2.py

In [ ]:
!python -m memory_profiler memscript2